<div style="display: flex; justify-content: space-between; align-items: flex-start;">
    <div style="text-align: left;">
        <p style="color:#FFD700; font-size: 15px; font-weight: bold; margin-bottom: 1px; text-align: left;">Published on  November 7, 2025</p>
        <h4 style="color:#4B0082; font-weight: bold; text-align: left; margin-top: 6px;">Author: Jocelyn C. Dumlao</h4>
        <p style="font-size: 17px; line-height: 1.7; color: #333; text-align: center; margin-top: 20px;"></p>
        <a href="https://www.linkedin.com/in/jocelyn-dumlao-168921a8/" target="_blank" style="display: inline-block; background-color: #003f88; color: #fff; text-decoration: none; padding: 5px 10px; border-radius: 10px; margin: 15px;">LinkedIn</a>
        <a href="https://github.com/jcdumlao14" target="_blank" style="display: inline-block; background-color: transparent; color: #059c99; text-decoration: none; padding: 5px 10px; border-radius: 10px; margin: 15px; border: 2px solid #007bff;">GitHub</a>
        <a href="https://www.youtube.com/@CogniCraftedMinds" target="_blank" style="display: inline-block; background-color: #ff0054; color: #fff; text-decoration: none; padding: 5px 10px; border-radius: 10px; margin: 15px;">YouTube</a>
        <a href="https://www.kaggle.com/jocelyndumlao" target="_blank" style="display: inline-block; background-color: #3a86ff; color: #fff; text-decoration: none; padding: 5px 10px; border-radius: 10px; margin: 15px;">Kaggle</a>
    </div>
</div>

# <div style="color:white;display:inline-block;border-radius:5px;background-color:#009688 ;font-family:Nexa;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;font-size:85%;letter-spacing:0.5px;margin:0;border: 6px groove #ffd700;"><b> </b>Imports Libraries</p></div>


In [ ]:
import os
import random
from glob import glob
import json
from pathlib import Path

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import cv2
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


import warnings
warnings.filterwarnings("ignore")

# <div style="color:white;display:inline-block;border-radius:5px;background-color:#009688 ;font-family:Nexa;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;font-size:85%;letter-spacing:0.5px;margin:0;border: 6px groove #ffd700;"><b> </b>Configuration</p></div>

In [ ]:
# Configuration

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

IMG_SIZE = 256
BATCH_SIZE = 8
EPOCHS = 10
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_PATH = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/"  # change if needed

print(f"Device: {DEVICE}")

# <div style="color:white;display:inline-block;border-radius:5px;background-color:#009688 ;font-family:Nexa;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;font-size:85%;letter-spacing:0.5px;margin:0;border: 6px groove #ffd700;"><b> </b>Dataset</p></div>

In [ ]:
# Dataset

class ForgeryDataset(Dataset):
    """Dataset that yields (image_tensor, mask_tensor) where mask is 1-channel binary."""
    def __init__(self, image_paths, mask_paths=None, is_authentic=None, img_size=IMG_SIZE):
        self.image_paths = list(image_paths)
        self.mask_paths = list(mask_paths) if mask_paths is not None else [None] * len(self.image_paths)
        self.is_authentic = list(is_authentic) if is_authentic is not None else [False] * len(self.image_paths)
        self.img_size = img_size

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (self.img_size, self.img_size)).astype(np.float32) / 255.0
        img = torch.from_numpy(img).permute(2, 0, 1).float()  # C,H,W

        if self.is_authentic[idx]:
            mask = np.zeros((self.img_size, self.img_size), dtype=np.float32)
        else:
            mask_path = self.mask_paths[idx]
            mask = np.load(mask_path)
            if mask.ndim == 3:
                mask = mask[0] if mask.shape[0] == 1 else mask[:, :, 0]
            mask = (mask > 0).astype(np.float32)
            mask = cv2.resize(mask, (self.img_size, self.img_size), interpolation=cv2.INTER_NEAREST)

        mask = torch.from_numpy(mask).unsqueeze(0).float()  # 1,H,W
        return img, mask


# <div style="color:white;display:inline-block;border-radius:5px;background-color:#009688 ;font-family:Nexa;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;font-size:85%;letter-spacing:0.5px;margin:0;border: 6px groove #ffd700;"><b> </b>Lightweight UNet</p></div>

In [ ]:
# Lightweight UNet

def conv_block(in_ch, out_ch):
    return nn.Sequential(
        nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
    )

class SmallUNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.enc1 = conv_block(3, 32)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = conv_block(32, 64)
        self.pool2 = nn.MaxPool2d(2)
        self.enc3 = conv_block(64, 128)

        self.up2 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec2 = conv_block(128, 64)
        self.up1 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.dec1 = conv_block(64, 32)

        self.out = nn.Conv2d(32, 1, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        e1 = self.enc1(x)
        p1 = self.pool1(e1)
        e2 = self.enc2(p1)
        p2 = self.pool2(e2)
        e3 = self.enc3(p2)

        u2 = self.up2(e3)
        cat2 = torch.cat([u2, e2], dim=1)
        d2 = self.dec2(cat2)

        u1 = self.up1(d2)
        cat1 = torch.cat([u1, e1], dim=1)
        d1 = self.dec1(cat1)

        out = self.out(d1)
        return self.sigmoid(out)


# <div style="color:white;display:inline-block;border-radius:5px;background-color:#009688 ;font-family:Nexa;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;font-size:85%;letter-spacing:0.5px;margin:0;border: 6px groove #ffd700;"><b> </b>RLE encode </p></div>

In [ ]:
# Utility: RLE encode 

def _rle_one(arr):
    """Encode a single 2D binary mask into RLE list of pairs."""
    dots = np.where(arr.T.flatten() == 1)[0]
    if len(dots) == 0:
        return []
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def rle_encode(masks, fg_val=1):
    return ';'.join(json.dumps(_rle_one(m)) for m in masks)


# <div style="color:white;display:inline-block;border-radius:5px;background-color:#009688 ;font-family:Nexa;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;font-size:85%;letter-spacing:0.5px;margin:0;border: 6px groove #ffd700;"><b> </b>Load file lists & split</p></div>

In [ ]:
# Load file lists & split 

forged_images = sorted(glob(os.path.join(DATA_PATH, 'train_images/forged/*.png')))
train_masks = sorted(glob(os.path.join(DATA_PATH, 'train_masks/*.npy')))
authentic_images = sorted(glob(os.path.join(DATA_PATH, 'train_images/authentic/*.png')))

matched_forged_images, matched_masks = [], []
for img_path in forged_images:
    img_name = os.path.basename(img_path).replace('.png', '')
    mask_path = os.path.join(DATA_PATH, f'train_masks/{img_name}.npy')
    if os.path.exists(mask_path):
        matched_forged_images.append(img_path)
        matched_masks.append(mask_path)

all_images = matched_forged_images + authentic_images
all_masks = matched_masks + [None] * len(authentic_images)
all_is_authentic = [False] * len(matched_forged_images) + [True] * len(authentic_images)

print(f"Total images: {len(all_images)} | Forged: {len(matched_forged_images)} | Authentic: {len(authentic_images)}")

# stratify safely
if len(set(all_is_authentic)) < 2:
    print("⚠️ Not enough class diversity — using non-stratified split.")
    stratify_arg = None
else:
    stratify_arg = all_is_authentic

train_imgs, val_imgs, train_masks_split, val_masks_split, train_auth, val_auth = train_test_split(
    all_images, all_masks, all_is_authentic,
    test_size=0.2, random_state=SEED, stratify=stratify_arg
)

train_ds = ForgeryDataset(train_imgs, train_masks_split, train_auth)
val_ds = ForgeryDataset(val_imgs, val_masks_split, val_auth)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print(f"✅ Dataloaders ready: {len(train_ds)} train samples, {len(val_ds)} val samples.")


# <div style="color:white;display:inline-block;border-radius:5px;background-color:#009688 ;font-family:Nexa;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;font-size:85%;letter-spacing:0.5px;margin:0;border: 6px groove #ffd700;"><b> </b>Training Utilities</p></div>

In [ ]:
# Training utilities

def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running = 0.0
    for imgs, masks in loader:
        imgs = imgs.to(device)
        masks = masks.to(device)
        preds = model(imgs)
        loss = criterion(preds, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running += loss.item()
    return running / len(loader)

@torch.no_grad()
def validate(model, loader, criterion, device):
    model.eval()
    running = 0.0
    for imgs, masks in loader:
        imgs = imgs.to(device)
        masks = masks.to(device)
        preds = model(imgs)
        loss = criterion(preds, masks)
        running += loss.item()
    return running / len(loader)

def pixel_f1(pred_mask, gt_mask):
    pred = (pred_mask > 0.5).astype(np.uint8).ravel()
    gt = (gt_mask > 0.5).astype(np.uint8).ravel()
    tp = np.sum((pred == 1) & (gt == 1))
    fp = np.sum((pred == 1) & (gt == 0))
    fn = np.sum((pred == 0) & (gt == 1))
    if tp + 0.5*(fp+fn) == 0:
        return 0.0
    return 2*tp / (2*tp + fp + fn)

# <div style="color:white;display:inline-block;border-radius:5px;background-color:#009688 ;font-family:Nexa;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;font-size:85%;letter-spacing:0.5px;margin:0;border: 6px groove #ffd700;"><b> </b>Instantiate Model, Loss, Optimizer</p></div>

In [ ]:
# Instantiate model, loss, optimizer

model = SmallUNet().to(DEVICE)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# <div style="color:white;display:inline-block;border-radius:5px;background-color:#009688 ;font-family:Nexa;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;font-size:85%;letter-spacing:0.5px;margin:0;border: 6px groove #ffd700;"><b> </b>Training Loop</p></div>

In [ ]:
# Training loop

train_losses, val_losses = [], []
print("🚀 Training started...")
for epoch in range(EPOCHS):
    tr_loss = train_one_epoch(model, train_loader, criterion, optimizer, DEVICE)
    val_loss = validate(model, val_loader, criterion, DEVICE)
    train_losses.append(tr_loss)
    val_losses.append(val_loss)
    print(f"Epoch {epoch+1}/{EPOCHS} — Train Loss: {tr_loss:.4f} | Val Loss: {val_loss:.4f}")

plt.figure(figsize=(6,4))
plt.plot(train_losses, label='train')
plt.plot(val_losses, label='val')
plt.title("Loss Curve")
plt.xlabel("Epoch")
plt.ylabel("BCE Loss")
plt.legend()
plt.tight_layout()
plt.show()



# <div style="color:white;display:inline-block;border-radius:5px;background-color:#009688 ;font-family:Nexa;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;font-size:85%;letter-spacing:0.5px;margin:0;border: 6px groove #ffd700;"><b> </b>Visualization</p></div>

In [ ]:
# visualization utilities

def _mask_contours(mask):
    mask_u8 = (mask.astype(np.uint8) * 255).astype(np.uint8)
    contours, _ = cv2.findContours(mask_u8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    result = []
    for c in contours:
        c = c.squeeze()
        if c.ndim == 1:
            continue
        result.append(c)
    return result

def overlay_mask_on_image(img_rgb, mask, color=(0,255,0), alpha=0.4):
    if img_rgb.dtype == np.float32 or img_rgb.dtype == np.float64:
        bg = (img_rgb * 255).astype(np.uint8).copy()
    else:
        bg = img_rgb.copy()
    overlay = bg.copy()
    cv2.drawContours(overlay, [c.astype(np.int32) for c in _mask_contours(mask)],
                     -1, color, thickness=-1)
    blended = cv2.addWeighted(overlay, alpha, bg, 1-alpha, 0)
    return blended

def visualize_predictions(model, image_paths, mask_paths, is_authentic, device=DEVICE, num_samples=6):
    model.eval()
    indices = list(range(len(image_paths)))
    chosen = random.sample(indices, min(num_samples, len(indices)))
    rows = len(chosen)
    plt.figure(figsize=(12, 4*rows))

    with torch.no_grad():
        for i, idx in enumerate(chosen):
            img_path = image_paths[idx]
            img = cv2.imread(img_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            orig_h, orig_w = img_rgb.shape[:2]

            img_resized = cv2.resize(img_rgb, (IMG_SIZE, IMG_SIZE)).astype(np.float32) / 255.0
            tensor = torch.from_numpy(img_resized).permute(2,0,1).unsqueeze(0).float().to(device)
            pred = model(tensor).cpu().numpy()[0,0]
            pred_resized = cv2.resize(pred, (orig_w, orig_h), interpolation=cv2.INTER_LINEAR)
            pred_binary = (pred_resized > 0.5).astype(np.uint8)

            if is_authentic[idx]:
                gt_mask = np.zeros((orig_h, orig_w), dtype=np.uint8)
                label = "AUTHENTIC"
            else:
                gt = np.load(mask_paths[idx])
                if gt.ndim == 3:
                    gt = gt[0] if gt.shape[0] == 1 else gt[:,:,0]
                gt = (gt > 0).astype(np.uint8)
                gt_mask = cv2.resize(gt, (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)
                label = "FORGED"

            forged_pixels = int(gt_mask.sum())
            forged_pct = forged_pixels / gt_mask.size * 100

            heat = cv2.applyColorMap((pred_resized*255).astype(np.uint8), cv2.COLORMAP_JET)
            heat = cv2.cvtColor(heat, cv2.COLOR_BGR2RGB)

            gt_overlay = overlay_mask_on_image(img_rgb, gt_mask, color=(0,255,0), alpha=0.35)
            pred_overlay = img_rgb.copy()
            contours = _mask_contours(pred_binary)
            if contours:
                cv2.drawContours(pred_overlay, contours, -1, (255,0,0), thickness=2)

            ax = plt.subplot(rows, 3, i*3 + 1)
            ax.imshow(img_rgb)
            ax.set_title(f"{label} - {os.path.basename(img_path)}")
            ax.axis("off")

            ax = plt.subplot(rows, 3, i*3 + 2)
            ax.imshow(gt_overlay)
            ax.set_title(f"Ground Truth — {forged_pixels:,} px ({forged_pct:.2f}%)")
            ax.axis("off")

            ax = plt.subplot(rows, 3, i*3 + 3)
            blended = (0.6 * img_rgb.astype(np.float32) + 0.4 * heat.astype(np.float32)).astype(np.uint8)
            if contours:
                cv2.drawContours(blended, contours, -1, (255,0,0), thickness=2)
            ax.imshow(blended)
            ax.set_title("Prediction heatmap + binary contour")
            ax.axis("off")

    plt.tight_layout()
    plt.show()


In [ ]:
# Visualize predictions

print("🔍 Visualizing some training samples...")
visualize_predictions(model, train_imgs, train_masks_split, train_auth, device=DEVICE, num_samples=6)

# <div style="color:white;display:inline-block;border-radius:5px;background-color:#009688 ;font-family:Nexa;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;font-size:85%;letter-spacing:0.5px;margin:0;border: 6px groove #ffd700;"><b> </b>Validation F1</p></div>

In [ ]:
# Validation F1

print("📏 Computing validation F1...")
model.eval()
val_f1s = []
with torch.no_grad():
    for idx in range(len(val_imgs)):
        img_path = val_imgs[idx]
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        orig_h, orig_w = img_rgb.shape[:2]
        img_resized = cv2.resize(img_rgb, (IMG_SIZE, IMG_SIZE)).astype(np.float32) / 255.0
        tensor = torch.from_numpy(img_resized).permute(2,0,1).unsqueeze(0).float().to(DEVICE)
        pred = model(tensor).cpu().numpy()[0,0]
        pred_resized = cv2.resize(pred, (orig_w, orig_h), interpolation=cv2.INTER_LINEAR)
        pred_binary = (pred_resized > 0.5).astype(np.uint8)

        if val_auth[idx]:
            gt_mask = np.zeros((orig_h, orig_w), dtype=np.uint8)
        else:
            gt = np.load(val_masks_split[idx])
            if gt.ndim == 3:
                gt = gt[0] if gt.shape[0] == 1 else gt[:,:,0]
            gt_mask = (gt > 0).astype(np.uint8)
            gt_mask = cv2.resize(gt_mask, (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)

        f1 = pixel_f1(pred_binary, gt_mask)
        val_f1s.append(f1)

print(f"✅ Mean pixel-wise F1 on validation: {np.mean(val_f1s):.4f}")


# <div style="color:white;display:inline-block;border-radius:5px;background-color:#009688 ;font-family:Nexa;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;font-size:85%;letter-spacing:0.5px;margin:0;border: 6px groove #ffd700;"><b> </b>Test Prediction & Submission </p></div>

In [ ]:
# Test prediction & submission 

print("🧾 Generating submission.csv...")
test_image_paths = sorted(glob(os.path.join(DATA_PATH, 'test_images/*.png')))
test_predictions = {}

with torch.no_grad():
    for img_path in test_image_paths:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        orig_h, orig_w = img.shape[:2]
        img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE)).astype(np.float32) / 255.0
        tensor = torch.from_numpy(img_resized).permute(2,0,1).unsqueeze(0).float().to(DEVICE)
        pred = model(tensor).cpu().numpy()[0,0]
        pred_resized = cv2.resize(pred, (orig_w, orig_h), interpolation=cv2.INTER_LINEAR)
        pred_binary = (pred_resized > 0.5).astype(np.uint8)

        image_id = os.path.splitext(os.path.basename(img_path))[0]
        if pred_binary.sum() == 0:
            test_predictions[image_id] = "authentic"
        else:
            rle = rle_encode([pred_binary], fg_val=1)
            test_predictions[image_id] = rle

submission_df = pd.DataFrame([{"case_id": k, "annotation": v} for k, v in test_predictions.items()])
submission_df.to_csv("submission.csv", index=False)
print("✅ submission.csv saved!")
print(submission_df.head())
